In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 10)

import numpy as np
#np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt
from matplotlib import cm

import ctapipe
from ctapipe.io.hessio import hessio_event_source
import pyhessio

import sys
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline")

import datapipe
import datapipe.denoising.wavelets_mrfilter as mrfilter

sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils")
import common_functions as common

from datapipe.io import images
from datapipe.io import geometry_converter
import datapipe.io.montecarlo_calibration_astri as mc_calibration

In [ ]:
tel_id = 1
#source = hessio_event_source("/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz", allowed_tels=[tel_id])
source = hessio_event_source("/Users/jdecock/data/gct/proton/group1run1000.simtel.gz", allowed_tels=[tel_id])

In [ ]:
for ev in source:
    if tel_id in ev.dl0.tel:
        print(ev.dl0.tel[tel_id+1].adc_sums[1])

In [ ]:
#print(ev)

In [ ]:
# Pedestal
#print(ev.mc.tel[tel_id].pedestal.shape)
print(ev.mc.tel[tel_id].pedestal[0])

# Gain
print(ev.mc.tel[tel_id].dc_to_pe[0])

# ADC sums
print(ev.r0.tel[tel_id].adc_sums[0])

print(ev.mc.tel[tel_id].photo_electron_image)

ev.mc.tel[tel_id].reference_pulse_shape.shape

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
plt.plot(ev.mc.tel[tel_id].reference_pulse_shape[0])

# TODO!!!
#help(pyhessio)
#pedestal, gains = get_mc_calibration_data(tel_id)
#print("pedestal:", pedestal)
#print("gains:", gains)

In [ ]:
# GET IMAGES ##############################################

pe_image = ev.mc.tel[tel_id].photo_electron_image   # 1D np array
calibrated_image = ev.r0.tel[tel_id].adc_sums     # ctapipe 0.4.0
pedestal = ev.mc.tel[tel_id].pedestal
gain = ev.mc.tel[tel_id].dc_to_pe
pixel_pos = ev.inst.pixel_pos[tel_id]

# calibrating
#calibrated_image = mc_calibration.apply_mc_calibration(uncalibrated_image, pedestal, gain)

In [ ]:
# %load ~/git/pub/jdhp-sap/snippets/ctapipe/plot_pixels_index.py
#!/usr/bin/env python3

"""
Display the pixel layout of the givent telescope as defined in a given simtel file.
"""

(pos_x_list, pos_y_list) = pixel_pos
pos_x_list = [float(pos.value) for pos in pos_x_list]
pos_y_list = [float(pos.value) for pos in pos_y_list]

assert len(pos_x_list) == len(pos_y_list)

# PLOT ####################################################################

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(13, 13))

# Scatter method ##################

ax.scatter(pos_x_list,    # x
           pos_y_list,    # y
           s=0.2,         # radius
           c="black",     # color
           alpha=0.75)

ax.plot(pos_x_list,    # x
        pos_y_list,    # y
        "k-",
        alpha=0.25)

for pixel_index in range(len(pos_x_list)):
    ax.text(pos_x_list[pixel_index], pos_y_list[pixel_index], str(pixel_index), fontsize=8)

ax.set_title("Pixels position (telescope {})".format(tel_id), fontsize=16)

ax.set_xlabel("x position (m)", fontsize=16)
ax.set_ylabel("y position (m)", fontsize=16)

ax.set_xlim(-0.17, 0.17)
ax.set_ylim(-0.17, 0.17)

plt.show()

### Plot image with ctapipe

In [ ]:
#%load ../../snippets/ctapipe/plot_events_image.py

import ctapipe
import ctapipe.visualization
from ctapipe.io.hessio import hessio_event_source

from matplotlib import pyplot as plt

channel = 0

pedestal = ev.mc.tel[tel_id].pedestal[channel]
gain = ev.mc.tel[tel_id].dc_to_pe[channel]
adc_sum = ev.r0.tel[tel_id].adc_sums[channel]
pe = ev.mc.tel[tel_id].photo_electron_image

#########################################################################

x, y = ev.inst.pixel_pos[tel_id]
foclen = ev.inst.optical_foclen[tel_id]
geom = ctapipe.io.CameraGeometry.guess(x, y, foclen)

disp = ctapipe.visualization.CameraDisplay(geom, title='CT%d' % tel_id)
disp.enable_pixel_picker()
disp.add_colorbar()

disp.axes.set_title('Telescope {:03d}'.format(tel_id))

# DISPLAY TIME-VARYING EVENT ############################################

#data = event.dl0.tel[tel_num].adc_samples[channel]
#for ii in range(data.shape[1]):
#    disp.image = data[:, ii]
#    disp.set_limits_percent(70)   # TODO
#    plt.savefig('CT{:03d}_EV{:05d}_S{:02d}.png'.format(tel_num, event_id, ii), bbox_inches='tight')

# DISPLAY INTEGRATED EVENT ##############################################

#image = pedestal
#image = gain
#image = adc_sum
image = pe

disp.image = pe

#disp.set_limits_minmax(0, 9000)
disp.set_limits_percent(70)        # TODO

# PLOT ##################################################################

plt.show()

## 2D to 1D

#### Plot 2D to 1D image map

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

img_map = np.zeros([8*6, 8*6], dtype=int)

# By default, pixels maps to the last element of input_img_ext (i.e. NaN)
img_map[:] = -1

# Map values
img_map[:8,8:-8] = np.arange(8*8*4).reshape([8,8*4])
img_map[8:40,:] = np.arange(32*48).reshape([32,48]) + 256
img_map[-8:,8:-8] = np.arange(8*8*4).reshape([8,8*4]) + 1792

plt.imshow(img_map, interpolation='nearest', origin='lower')
plt.colorbar()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

input_img_ext = np.zeros(image.shape[0] + 1)
input_img_ext[:-1] = image[:]
input_img_ext[-1] = np.nan

img_2d = input_img_ext[[img_map.ravel()]].reshape([8*6, 8*6])

plt.imshow(img_2d, interpolation='nearest', origin='lower')
plt.colorbar()

#### Plot 2D pixel mask

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))

img_mask = np.zeros([8*6, 8*6], dtype=int)

img_mask[:8,8:-8] = 1
img_mask[8:40,:] = 1
img_mask[-8:,8:-8] = 1

plt.imshow(img_mask, interpolation='nearest', origin='lower')
plt.colorbar()
plt.show()